# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119473e+02     1.692437e+00
 * time: 0.09222006797790527
     1     1.096474e+01     9.110164e-01
 * time: 0.9539239406585693
     2    -1.264092e+01     1.079850e+00
 * time: 1.114220142364502
     3    -3.438456e+01     8.115111e-01
 * time: 1.254073143005371
     4    -4.809435e+01     6.014056e-01
 * time: 1.412196159362793
     5    -5.711703e+01     1.848837e-01
 * time: 1.5606160163879395
     6    -5.984830e+01     1.003379e-01
 * time: 1.6691160202026367
     7    -6.085386e+01     5.540393e-02
 * time: 1.78391695022583
     8    -6.124190e+01     7.006512e-02
 * time: 1.8926260471343994
     9    -6.156455e+01     3.548090e-02
 * time: 2.008923053741455
    10    -6.178687e+01     2.895165e-02
 * time: 2.1198880672454834
    11    -6.195798e+01     2.006668e-02
 * time: 2.2354531288146973
    12    -6.202562e+01     1.993440e-02
 * time: 2.3452789783477783
    13    -6.209630e+01     1.787633e-02
 * time: 2.459972143173217

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671056
    AtomicLocal         -18.8557704
    AtomicNonlocal      14.8522685
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336818

    total               -62.261666458484
